In [7]:
import pandas as pd
import pandas as pd
import clickhouse_driver as ch
from hw import HoltWinters

df = pd.read_csv('test.csv').drop('Unnamed: 0', axis=1)

In [19]:
import pandas as pd
import clickhouse_driver as ch
from hw import HoltWinters

with open('./queries/q.sql', 'r') as f:
    q = f.read()
q=q.replace('${NAME}', 'RU | Потенция')
sqlCH = ch.Client(host="192.168.235.72",
                    user="valferov",
                    password="bUfMupl1kDgK1337")

df = pd.DataFrame(sqlCH.execute(q))

In [15]:
with open('./queries/q_campaignId.sql', 'r') as f:
    q = f.read()
q=q.replace('${CAMPAIGNS_ID}', '14569')
sqlCH = ch.Client(host="192.168.235.72",
                    user="valferov",
                    password="bUfMupl1kDgK1337")

df = pd.DataFrame(sqlCH.execute(q))
df[0][0]

'RU | Амулет'

In [20]:
df.columns = ['datetime', 'shows', 'campaings']

test = df.sort_values('datetime')
test = test.drop(test.loc[test['shows'] == 0].index)\
            .drop(test.index.values[-5:])\
            .reset_index().drop('index', axis=1)
test['datetime'] = pd.to_datetime(test['datetime'])
test

,datetime,shows,campaings
0,2022-07-19 09:00:00,369755,RU | Потенция
1,2022-07-19 09:00:00,374665,RU | Потенция
2,2022-07-19 10:00:00,398972,RU | Потенция
3,2022-07-19 10:00:00,278363,RU | Потенция
4,2022-07-19 11:00:00,310807,RU | Потенция
...,...,...,...
630,2022-08-01 12:00:00,81843,RU | Потенция
631,2022-08-01 12:00:00,597094,RU | Потенция
632,2022-08-01 13:00:00,84220,RU | Потенция
633,2022-08-01 13:00:00,645279,RU | Потенция


In [23]:
test['datetime'].values[-1]

numpy.datetime64('2022-08-01T14:00:00.000000000')

In [5]:
from scipy.optimize import minimize
from cross_val import CVScore

x=[1,0,1]

timeseriesCV = CVScore(test['shows'])

opt = minimize(timeseriesCV.timeseriesCVscore, x0=x, method='Nelder-Mead', 
                bounds=((0, 1), (0, 1), (0, 1)), options={'maxiter': 10000})

opt

 final_simplex: (array([[1.        , 0.00647881, 1.        ],
       [1.        , 0.00647881, 1.        ],
       [1.        , 0.00647881, 1.        ],
       [1.        , 0.00647881, 1.        ]]), array([9.65488063e+10, 9.65488063e+10, 9.65488063e+10, 9.65488063e+10]))
           fun: 96548806342.67822
       message: 'Optimization terminated successfully.'
          nfev: 288
           nit: 123
        status: 0
       success: True
             x: array([1.        , 0.00647881, 1.        ])

In [14]:
df

,datetime,shows,campaings
0,2022-07-27 10:00:00,553373,RU | Гидрактив
1,2022-07-28 08:00:00,613554,RU | Гидрактив
2,2022-07-26 21:00:00,536314,RU | Гидрактив
3,2022-07-19 19:00:00,230898,RU | Гидрактив
4,2022-07-27 03:00:00,215257,RU | Гидрактив
...,...,...,...
278,2022-07-21 18:00:00,353819,RU | Гидрактив
279,2022-07-26 13:00:00,742657,RU | Гидрактив
280,2022-07-19 09:00:00,374665,RU | Гидрактив
281,2022-07-22 06:00:00,226570,RU | Гидрактив


# Build model & validation

In [6]:
hw = HoltWinters(test['shows'], slen=24, alpha=opt.x[0], 
                    beta=opt.x[1], gamma=opt.x[2], n_preds=0,
                    scaling_factor=2.56)
hw.triple_exponential_smoothing()

validation = pd.DataFrame(dict(shows=test['shows'],
                                datetime=test['datetime'],
                                predict=hw.result))
validation

validation['SSR'] = (validation['shows'] - validation['predict'])**2
validation['SST'] = (validation['shows'] - validation['shows'].mean())**2
check = validation['SSR'].sum()/validation['SST'].sum()
check

0.00015195435852972998

# Forecast

In [24]:
hw_forecast = HoltWinters(test['shows'], slen=24, alpha=opt.x[0], 
                    beta=opt.x[1], gamma=opt.x[2], n_preds=10,
                    scaling_factor=2.56)
hw_forecast.triple_exponential_smoothing()

forecast_arr = []
for i in range(int(len(test)/3)):
    forecast_arr.append(test['datetime'].values[-1]+pd.Timedelta(hours=i))
forecast = pd.DataFrame(forecast_arr, columns=['datetime'])
forecast['forecast'] = hw_forecast.result[-len(forecast['datetime']):]
forecast

,datetime,forecast
0,2022-08-01 14:00:00,56120.691803
1,2022-08-01 15:00:00,576885.199851
2,2022-08-01 16:00:00,63411.734751
3,2022-08-01 17:00:00,595425.153769
4,2022-08-01 18:00:00,595863.718056
...,...,...
206,2022-08-10 04:00:00,164701.580327
207,2022-08-10 05:00:00,168381.273202
208,2022-08-10 06:00:00,253932.043000
209,2022-08-10 07:00:00,129735.082028


In [8]:
predict = pd.concat([test[['datetime', 'shows']], 
                        forecast[['datetime', 'forecast']]],
                    ignore_index=True)
predict.loc[predict['shows'].isna() == True]['forecast'].sum()

55424617.14298199

In [14]:
predict.to_csv('test.csv')

# Build plot

In [10]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(x=test['datetime'].values, 
                        y=test['shows'].values,
                        name=f'campaign'))
fig.add_trace(go.Scatter(x=test['datetime'].values,
                            y=hw.result, name='predict'))
fig.add_trace(go.Scatter(x=forecast['datetime'].values, 
                            y=forecast['forecast'].values,
                            name=f'forecast for {50} values'))
fig.update_layout(
    font_family="Courier New",
    title_text=f'Forecast for campaign with accurancy {round(100-check, 10)}%'
)
fig.show()

In [18]:
df

,0
0,RU | Амулет


In [69]:
import plotly.graph_objects as go

forecastPeriod=10

forecast = exp.forecast(forecastPeriod)

fig = go.Figure()

fig.add_trace(go.Scatter(x=test['datetime'].values, 
                        y=test['shows'].values,
                        name=f'Campaigns {campaigns[0]}'))

fig.add_trace(go.Scatter(x=[test['datetime'].values[-1] + pd.Timedelta(hours=i) 
                                for i in range(forecastPeriod)],
                            y=forecast, 
                            name=f'Forecast for {forecastPeriod} hours'))

fig.show()